In [1]:
# Imports
import pandas as pd
import numpy as np
import tarfile

In [ ]:
tar_path = '../Data/Raw/yelp_dataset.tar'

# List all files in the tar archive
with tarfile.open(tar_path, 'r') as tar:
    print("Files in the tar archive:")
    for member in tar.getmembers():
        print(f"  - {member.name} ({member.size / 1024 / 1024:.1f} MB)")

In [2]:
import pandas as pd

data_path = '../Data/Raw/'

# Load all JSON files into DataFrames
print("Loading datasets... this may take a while\n")

# print("Loading business data...")
# business_df = pd.read_json(f'{data_path}yelp_academic_dataset_business.json', lines=True)
# print(f"✓ business_df: {len(business_df):,} rows")
#
# print("\nLoading checkin data...")
# checkin_df = pd.read_json(f'{data_path}yelp_academic_dataset_checkin.json', lines=True)
# print(f"✓ checkin_df: {len(checkin_df):,} rows")

print("\nLoading review data (this is BIG - 5GB)...")
review_df = pd.read_json(f'{data_path}yelp_academic_dataset_review.json', lines=True)
print(f"✓ review_df: {len(review_df):,} rows")

# print("\nLoading tip data...")
# tip_df = pd.read_json(f'{data_path}yelp_academic_dataset_tip.json', lines=True)
# print(f"✓ tip_df: {len(tip_df):,} rows")
#
# print("\nLoading user data (also BIG - 3GB)...")
# user_df = pd.read_json(f'{data_path}yelp_academic_dataset_user.json', lines=True)
# print(f"✓ user_df: {len(user_df):,} rows")

print("\n All datasets loaded!")

Loading datasets... this may take a while


Loading review data (this is BIG - 5GB)...
✓ review_df: 6,990,280 rows

 All datasets loaded!
